In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import sys
import ROOT
from ROOT import TFile, TH1D, TH2D, TCanvas, gStyle, TLine, TTree
from ROOT import (RooArgSet, RooRealVar, RooDataSet, RooPlot, RooFit, RooStats, RooArgList)

import math
import itertools
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from root_pandas import read_root
import pandas as pd

sys.path.append('/home/vmueller/repos')
from dopy.doroot.root_utils import read_roodataset_from_tree, plot_pulls
from dopy.doplot.plotting import Plotter, Plot                

In [2]:

from ROOT import RooRealVar, RooExponential, RooExtendPdf, RooAddPdf, RooArgList, RooGaussian
from ROOT import RooFormulaVar
from ROOT import gSystem
gSystem.Load('libRooFit.so')
gSystem.Load('/home/chasenberg/repos/dopy/dolib/CustomShapes/libKll')
from ROOT import RooIpatia2

from ROOT import gSystem
gSystem.Load("libRooFit.so")
gSystem.Load("/home/vmueller/repos/bd2jpsieeks/external/DecRateCoeff/libDecRateCoeff.so")
from ROOT import DecRateCoeff_Bd,RooUnblindUniform

In [5]:


n_events = 10000
save_path = 'plots_root/'

x = ROOT.RooRealVar('x', 'variable', -10, 10, 'MeV/#it{c}^{2}')
mean = ROOT.RooRealVar('mean', 'mean', 0, -2, 2)
sigma = ROOT.RooRealVar('sigma', 'sigma', 2, 1, 5)

gauss1 = ROOT.RooGaussian('gauss', 'gauss', x, mean, sigma)

# Create pseudodata and fit it
data = gauss1.generate(ROOT.RooArgSet(x), n_events)


#Define fitmodel with blinded mean and sigma


sigma = ROOT.RooRealVar('sigma', 'sigma', 2, 1, 5)
sigma_ub = RooUnblindUniform('sigma_ub','sigma_ub', 'testing', 2, sigma)
mean2 = ROOT.RooRealVar('mean', 'mean', 0, -2, 2)
mean_ub = RooUnblindUniform('mean_ub','mean_ub', 'adfbeee', 2, mean2)

gauss = ROOT.RooGaussian('gauss', 'gauss', x, mean_ub, sigma_ub)

#fit model with blinded paramters to generated data 
gauss.fitTo(data)

# Prepare frame
frame = x.frame(ROOT.RooFit.Bins(40))

data.plotOn(frame, ROOT.RooFit.Name("data"))
gauss.plotOn(frame, ROOT.RooFit.Name("fit"))

### Add additional informations to the plot

# Create TLegend
legend = ROOT.TLegend(0.7, 0.7, 0.85, 0.85, 'Legend')
legend.AddEntry(frame.findObject('data'), 'Data points', 'p');
legend.AddEntry(frame.findObject('fit'), 'Gauss fit', 'l')

# Create TLatex
text_size = 0.045
latex = ROOT.TLatex(0.2, 0.78, '#splitline{Toy MC example}' + \
                                 '{#it{f}(x) = #frac{1}{#sigma #sqrt{2 #pi}}' + \
                                 ' e^{- 0.5 #left( #frac{x - #mu}{#sigma} #right)^{2}}}')
latex.SetTextSize(text_size*0.75)
latex.SetNDC()

# Set y-axis minimum to a positive value (to remove the overlapping zero)
frame.GetYaxis().SetRangeUser(0.1, frame.GetMaximum()*1.2)

# Plot simple (== without pulls) using plot_simple or with pulls using plot_pulls
#can, _ = plot_simple('gaussian_fit_simple', frame, drawables=[latex, legend],
#                      save_path=save_path)
can, _ = plot_pulls('gaussian_fit_pulls', frame, drawables=[latex, legend], 
                    save_path=save_path)
can  # To display plot in notebooks

TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: can
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: pulls_all (Potential memory leak).
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: pulls_ok (Potential memory leak).
TROOT::Append:0: RuntimeWarning: Replacing existing TH1: pulls_danger (Potential memory leak).


<ROOT.TCanvas object ("can") at 0x74af020>